<a href="https://colab.research.google.com/github/JaeminKIM-Irene/FC_AI_ML_Project/blob/main/irenekim/ML_Project_ExtraTrees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 설치에 필요한 라이브러리들이 있다면 모두 적어둡니다. anaconda에 기본적으로 설치되지 않은 라이브러리들을 적어두세요.
!pip install optuna

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import ExtraTreesClassifier

from sklearn.model_selection import KFold

import optuna

from sklearn.model_selection import StratifiedKFold
from functools import partial
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer


from sklearn.metrics import roc_auc_score

In [ ]:
# google drive mount
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# read data
result_list = []
base_url = './playground-series-s3e23/'
train = pd.read_csv(base_url+'train.csv', index_col='id')
test = pd.read_csv(base_url+'test.csv', index_col='id')
submission = pd.read_csv(base_url+'sample_submission.csv', index_col='id')

with pd.option_context('display.min_rows', 6) :
  display(train)

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
id,,,,,,,,,,,,,,,,,,,,,
0,22.0,3.0,1.0,2.0,60.0,278.63,0.06,19.56,14.25,5448.79,...,17,1,1,0,16.0,9.0,38.0,22.0,5.0,False
1,14.0,2.0,1.0,2.0,32.0,151.27,0.14,7.00,21.11,936.71,...,11,0,1,0,11.0,11.0,18.0,14.0,3.0,False
2,11.0,2.0,1.0,2.0,45.0,197.65,0.11,8.05,22.76,1754.01,...,8,0,1,0,12.0,11.0,28.0,17.0,3.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101760,26.0,1.0,1.0,1.0,83.0,360.17,0.04,22.75,12.56,5893.69,...,20,0,4,0,10.0,8.0,51.0,28.0,1.0,False
101761,10.0,2.0,1.0,2.0,43.0,191.76,0.15,6.46,29.53,1534.04,...,7,0,1,0,11.0,14.0,24.0,19.0,3.0,False
101762,136.0,18.0,18.0,1.0,296.0,1704.57,0.02,44.82,44.08,77011.02,...,102,1,11,0,22.0,31.0,162.0,135.0,35.0,False


In [ ]:
# vif
selected_col = ['loc', 'v(g)', 'ev(g)', 'iv(g)', 'v', 'l', 'd', 'i', 'e', 't', 'lOComment', 'lOBlank', 'locCodeAndComment', 'uniq_Opnd']

In [ ]:
# data split for baseline model
from sklearn.model_selection import train_test_split

X = train.drop(columns=['defects'])
#X = train[selected_col]
y = train.defects

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=61)

In [ ]:
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(81410, 21) (81410,) (20353, 21) (20353,)


## Baseline Model

## Optuna

In [ ]:
def optimizer(trial, X, y, K) :
  min_samples_split = trial.suggest_int('min_samples_split', 2, 12)
  min_impurity_decrease= trial.suggest_float('min_impurity_decrease',0.0, 0.01)
  n_estimators = trial.suggest_int('n_estimators', 50, 400)
  max_depth = trial.suggest_int('max_depth', 3, 9)
  max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 15, 25)
  criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
  max_features=trial.suggest_uniform('max_features', 0.1, 1.0)
  min_weight_fraction_leaf=trial.suggest_uniform('min_weight_fraction_leaf', 0.00001,0.4)
  # n_estimators = trial.suggest_int('n_estimators', 50, 120)
  # max_depth = trial.suggest_int('max_depth', 10, 16)
  # max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 15, 25)
  # criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])


  model = make_pipeline(FunctionTransformer(np.log1p),
                        ExtraTreesClassifier(n_estimators = n_estimators,
                                            max_depth = max_depth,
                                            max_leaf_nodes = max_leaf_nodes,
                                            criterion = criterion,
                                            min_impurity_decrease=min_impurity_decrease,
                                            class_weight= "balanced",
                                            min_samples_split=min_samples_split,
                                            min_samples_leaf=min_samples_leaf,
                                            max_features=max_features,
                                            min_weight_fraction_leaf=min_weight_fraction_leaf,
                                            bootstrap=False,
                                            random_state=61
                                             ))

  # model = make_pipeline(FunctionTransformer(np.log1p),
  #                       ExtraTreesClassifier(n_estimators = n_estimators,
  #                                           max_depth = max_depth,
  #                                           max_leaf_nodes = max_leaf_nodes,
  #                                           criterion = criterion,
  #                                           random_state=61
  #                                            ))

  folds = StratifiedKFold(shuffle=True, random_state=61)
  oof = np.full(len(train), np.nan)
  auc_list = []

  for train_idx, val_idx in folds.split(X, y) :
    X_train = X.iloc[train_idx, :]
    y_train = y.iloc[train_idx]

    X_val = X.iloc[val_idx, :]
    y_val = y.iloc[val_idx]

    model.fit(X_train, y_train)

    try :
      y_val_pred = model.predict_proba(X_val)[:, 1]
    except :
      y_val_pred = model.decision_function(X_val)
    oof[val_idx] = y_val_pred
    auc = roc_auc_score(y_val, y_val_pred)
    auc_list.append(auc)

  return np.mean(auc_list)

In [ ]:
K = 10
opt_func = partial(optimizer, X=X_train, y=y_train, K=K)

study = optuna.create_study(direction='maximize')
study.optimize(opt_func, n_trials=50)

[I 2023-10-17 18:44:16,998] A new study created in memory with name: no-name-7a388d46-aec1-479d-9027-2ed0328c2168
C:\Users\TT21\AppData\Local\Temp\ipykernel_29956\2789354698.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  max_features=trial.suggest_uniform('max_features', 0.1, 1.0)
C:\Users\TT21\AppData\Local\Temp\ipykernel_29956\2789354698.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  min_weight_fraction_leaf=trial.suggest_uniform('min_weight_fraction_leaf', 0.00001,0.4)
[I 2023-10-17 18:44:20,318] Trial 0 finished with value: 0.7432984752374863 and parameters: {'min_samples_split': 3, 'min_impurity_decrease': 0.002241357197815126, 'n_estimators': 52, 'max_depth': 7, 'max_leaf_nodes': 19, 'cri

In [ ]:
# boosting type = dart

In [ ]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_colsample_bynode,params_learning_rate,params_max_depth,params_n_estimators,params_reg_lambda,state
0,0,0.788587,2023-10-17 14:24:36.731905,2023-10-17 14:24:41.193551,0 days 00:00:04.461646,0.658321,0.152315,6,167,3.484973,COMPLETE
1,1,0.769548,2023-10-17 14:24:41.194549,2023-10-17 14:24:48.292626,0 days 00:00:07.098077,0.632092,0.295517,12,69,3.293869,COMPLETE
2,2,0.780958,2023-10-17 14:24:48.294620,2023-10-17 14:25:21.760992,0 days 00:00:33.466372,0.752861,0.037676,14,191,1.761860,COMPLETE
3,3,0.781131,2023-10-17 14:25:21.762986,2023-10-17 14:25:27.354850,0 days 00:00:05.591864,0.566693,0.208899,7,184,1.123674,COMPLETE
4,4,0.766183,2023-10-17 14:25:27.356848,2023-10-17 14:25:42.754339,0 days 00:00:15.397491,0.676987,0.228759,14,128,4.172292,COMPLETE
5,5,0.766616,2023-10-17 14:25:42.756334,2023-10-17 14:25:53.227673,0 days 00:00:10.471339,0.672251,0.209655,14,62,0.853603,COMPLETE
6,6,0.776758,2023-10-17 14:25:53.229668,2023-10-17 14:26:01.622028,0 days 00:00:08.392360,0.731293,0.143317,9,200,0.661321,COMPLETE
7,7,0.791010,2023-10-17 14:26:01.623026,2023-10-17 14:26:04.960061,0 days 00:00:03.337035,0.746368,0.116083,6,123,3.383253,COMPLETE
8,8,0.791521,2023-10-17 14:26:04.962055,2023-10-17 14:26:13.196171,0 days 00:00:08.234116,0.545177,0.027362,8,187,2.420335,COMPLETE
9,9,0.767131,2023-10-17 14:26:13.198166,2023-10-17 14:26:20.156556,0 days 00:00:06.958390,0.799091,0.290478,12,65,2.138494,COMPLETE


In [ ]:
print('Best Score: %.4f' % study.best_value)
print('Best Params: ', study.best_trial.params)

Best Score: 0.7856
Best Params:  {'min_samples_split': 3, 'min_impurity_decrease': 0.0001173921525417284, 'n_estimators': 254, 'max_depth': 9, 'max_leaf_nodes': 21, 'criterion': 'entropy', 'min_samples_leaf': 1, 'max_features': 0.908533864618086, 'min_weight_fraction_leaf': 0.03076428622124373}


In [ ]:
# few paramters
print('Best Score: %.4f' % study.best_value)
print('Best Params: ', study.best_trial.params)

Best Score: 0.7801
Best Params:  {'n_estimators': 112, 'max_depth': 12, 'max_leaf_nodes': 25, 'criterion': 'entropy'}


In [ ]:
# fix learning rate 0.02051093484555017, max_depth 5
print('Best Score: %.4f' % study.best_value)
print('Best Params: ', study.best_trial.params)

Best Score: 0.7927
Best Params:  {'colsample_bynode': 0.606710008819992, 'reg_lambda': 4.1260993488428355, 'n_estimators': 180}


In [ ]:
# split 0.2
print('Best Score: %.4f' % study.best_value)
print('Best Params: ', study.best_trial.params)

Best Score: 0.7855
Best Params:  {'min_samples_split': 8, 'min_impurity_decrease': 6.23657973379539e-05, 'n_estimators': 85, 'max_depth': 9, 'max_leaf_nodes': 17, 'criterion': 'entropy', 'min_samples_leaf': 3, 'max_features': 0.7212064973323903, 'min_weight_fraction_leaf': 0.02791190362320881}


In [ ]:
# fix learning rate 0.02051093484555017, max_depth 5, n_estimator 180, reg_lambda 4.380047157070892
print('Best Score: %.4f' % study.best_value)
print('Best Params: ', study.best_trial.params)

Best Score: 0.7926
Best Params:  {'colsample_bynode': 0.5720860013558229}


In [ ]:
# xgboost + vif
print('Best Score: %.4f' % study.best_value)
print('Best Params: ', study.best_trial.params)

Best Score: 0.7920
Best Params:  {'max_depth': 5, 'colsample_bynode': 0.5031839611117191, 'reg_lambda': 2.52415125709357, 'learning_rate': 0.05888255729980939, 'n_estimators': 105}


In [ ]:
# xgboost + vif + fix max_depth 5
print('Best Score: %.4f' % study.best_value)
print('Best Params: ', study.best_trial.params)

Best Score: 0.7920
Best Params:  {'colsample_bynode': 0.5837233209049923, 'reg_lambda': 4.364543135820495, 'learning_rate': 0.0428862043022044, 'n_estimators': 183}


In [ ]:
# xgboost + vif + fix max_depth 5 + learning_rate 0.0428862043022044
print('Best Score: %.4f' % study.best_value)
print('Best Params: ', study.best_trial.params)

Best Score: 0.7921
Best Params:  {'colsample_bynode': 0.5356344306876495, 'reg_lambda': 4.246322702366835, 'n_estimators': 117}


In [ ]:
# xgboost + vif + fix max_depth 5 + learning_rate 0.0428862043022044 + n_estimators 117
print('Best Score: %.4f' % study.best_value)
print('Best Params: ', study.best_trial.params)

Best Score: 0.7921
Best Params:  {'colsample_bynode': 0.5087883688153395, 'reg_lambda': 4.904855710479306}


In [ ]:
# k 10
print('Best Score: %.4f' % study.best_value)
print('Best Params: ', study.best_trial.params)

Best Score: 0.7849
Best Params:  {'min_samples_split': 5, 'min_impurity_decrease': 3.0203553124130877e-05, 'n_estimators': 85, 'max_depth': 4, 'max_leaf_nodes': 16, 'criterion': 'entropy', 'min_samples_leaf': 1, 'max_features': 0.7054523869071353, 'min_weight_fraction_leaf': 0.0009702228455310581}


In [ ]:
# hyper-parameter들의 중요도
optuna.visualization.plot_param_importances(study)

## Test & Submission file

In [ ]:
def oof_preds(best_model) :
  folds = StratifiedKFold(shuffle=True, random_state=61)
  final_preds = []
  auc_list = []

  for i, (train_idx, val_idx) in enumerate(folds.split(X, y)) :
    X_train = X.iloc[train_idx, :]
    y_train = y.iloc[train_idx]
    X_val = X.iloc[val_idx, :]
    y_val = y.iloc[val_idx]

    print(f"========== Fold {i+1} ==========")
    best_model.fit(X_train, y_train)
    preds = best_model.predict_proba(X_val)[:, 1]
    test_preds = best_model.predict_proba(test)[:, 1]
    final_preds.append(test_preds)
    auc = roc_auc_score(y_val, preds)

    auc_list.append(auc)

  print(f'AUC :', np.mean(auc))
  return final_preds

In [ ]:
best_params = study.best_trial.params

best_model = LGBMClassifier(**best_params,
                            random_state=61)

preds = oof_preds(best_model)
preds = np.mean(preds, axis=0)

preds

========== Fold 1 ==========
[LightGBM] [Info] Number of positive: 18451, number of negative: 62959
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.030658 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3547
[LightGBM] [Info] Number of data points in the train set: 81410, number of used features: 21
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.226643 -> initscore=-1.227365
[LightGBM] [Info] Start training from score -1.227365
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

array([0.23844273, 0.18092279, 0.65751052, ..., 0.17297542, 0.09837831,
       0.83926669])

In [ ]:
submission.defects = preds

In [ ]:
submission.to_csv('submission_lightGBM_is_balance.csv')